In [1]:
# helper library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os, re, cv2
import json, shutil
import collections
import base64,random
from tqdm import tqdm 
from typing import Dict, List
import matplotlib.pyplot as plt
from collections import defaultdict
from glob import glob
plt.style.use("seaborn")
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns',50)
from numpyencoder import NumpyEncoder
from numpy import random

# Image
from PIL import Image,ImageDraw
import cv2 as cv


# Function

In [2]:
# dataset의 개수를 파악하는 function
def number_of_data(data_path,number_of_image = 0 ,number_of_annotation = 0,total_data = 0):
    '''
    data_path: data directory path
    number_of_image: number of image
    number_of_annotation: number of anntation
    '''
    for file in os.listdir(data_path):
        path = os.path.join(data_path+'/',file)
        if os.path.isfile(path):
            total_data +=1
            if file.endswith('json'):
                number_of_annotation += 1
            elif file.endswith('png'):
                number_of_image += 1
        elif os.path.isdir(path) :
            number_of_image,number_of_annotation,total_data = number_of_data(path,number_of_image,number_of_annotation,total_data)
    return number_of_image,number_of_annotation,total_data

In [3]:
def get_annotation(data_path,annotation_list=[]):
    '''
    data_path: data directory path
    annotation_list: list of annotaion(label)
    '''
    for file in os.listdir(data_path):
        path = os.path.join(data_path+'/',file)
        if os.path.isfile(path):
            if file.endswith('json'):
                with open(path,'r',encoding="UTF-8") as annotation:
                    annotation_list.append(json.load(annotation))
        elif os.path.isdir(path) :
            get_annotation(path,annotation_list)
    return annotation_list

In [4]:
def annotation_summary(annotations):
    '''
    annotations: annotation of datas
    
    size : image width,height
    sun_day: data annotaion
    '''
    size = []
    sun_day = 0
    train_annotation = 0
    validation_annotation = 0
    test_annotation = 0
    attribute = []
    label = []
    for annotation in annotations:
        size.append(annotation['size'])
        if annotation['weather'] == 'sunny' and annotation['time'] == 'day':
            sun_day +=1

            if annotation['data_purpose'] == 'train':
                train_annotation += 1
            elif annotation['data_purpose'] == 'validation':
                validation_annotaion += 1
            else :
                test_annotation += 1
                
            for annotaion_ in annotation['annotations']:
                label.append(annotaion_['label'])
                attribute.append(annotaion_ ['attribute'])
                
    print('size of image',set(size))
    print('number of label',len(set(label)))
    print('number of class',len(set(attribute)))
    print('number of sunny and day image:', sun_day)
    print('number of train annotation:', train_annotation)
    print('number of validation annotation:', validation_annotation)
    print('number of test annotation:', test_annotation)
    
    return size,label,attribute

In [5]:
# annotaion만 따로 list로 저장하는 함수
def move_file(data_path,move_list=[],move_path = 'C:/Users/Lee/Desktop/AIFFEL/Test'):
    '''
    data_path: data directory path
    annotation_list: list of annotaion(label)
    '''
    for file in os.listdir(data_path):
        path = os.path.join(data_path+'/',file)
        if os.path.isfile(path):
            move_list.append(path)                             
        elif os.path.isdir(path) :
            move_file(path,move_list)
    for i in move_list:
        shutil.copy(i,move_path) 
    
    return move_list

In [6]:
def change_file_name(data_path):
    for file in os.listdir(data_path):
        if file.endswith('json'):
            shutil.move(data_path+'/'+file,data_path+'/'+file[:-13]+'.json') 

In [7]:
def adjust_annotation(path):
    count=0
    for file in os.listdir(path):
        if file.endswith('json'):
            id  = []
            with open(path+'/'+file, 'r', encoding="UTF-8") as annotations:
                anno = json.load(annotations)
                for index,annotation in enumerate(anno['annotations']):
                    if (annotation['label'] == 'traffic_sign') or (annotation['label'] == 'traffic_light') or (annotation['label'] == 'ptw') or (annotation['label'] == 'rider') or (annotation['label'] == 'car_number') or (annotation['label'] == 'movable_object') or (annotation['label'] == 'unknown'):
                        id.append(index)
                    elif annotation['area']<625 :
                        id.append(index)

                id.reverse()
                for index in id:
                    del anno['annotations'][index]
            try:
                with open(path+'/'+file, 'w', encoding='utf-8') as ch_annotations:
                    json.dump(anno, ch_annotations, indent="\t")
            except:
                count+=1
    return count

In [8]:
def make_class(path):
    count = 0
    for file in os.listdir(path):
        if file.endswith('json'):
            id  = []
            with open(path+'/'+file, 'r', encoding="UTF-8") as annotations:
                anno = json.load(annotations)

                for index,annotation in enumerate(anno['annotations']):
                    if (annotation['label'] == 'face') or (annotation['label'] == 'pedestrian'):
                        annotation['class'] = 'pedestrian'
                    elif annotation['attribute'] == 'car':
                        annotation['class'] = 'car'
                    elif (annotation['attribute'] == 'bus_l') or(annotation['attribute'] == 'bus_s'):
                        annotation['class'] = 'bus'
                    elif (annotation['attribute'] == 'truck_l') or(annotation['attribute'] == 'truck_s'):
                        annotation['class'] = 'truck'
                    else:
                        annotation['class'] = 'other'
                   
            try:
                with open(path+'/'+file, 'w', encoding='utf-8') as ch_annotations:
                    json.dump(anno, ch_annotations, indent="\t")
            except:
                count+=1
    return count

In [9]:
def split_image_annotation(root_path,image_path,annotation_path):
    for file in os.listdir(root_path):
        if file.endswith('png'):
            shutil.move(root_path+file,image_path)
        elif file.endswith('json'):
            shutil.move(root_path+file,annotation_path)

In [10]:
def json_to_txt(json_path = 'C:/Users/Lee/Desktop/AIFFEL/Test/annotation', mov_path = 'C:/Users/Lee/Desktop/AIFFEL/Test/annotation_txt'):
    '''
    json_path: json file 경로
    mov_path: txt 파일들 생성 경로 -> 디렉토리 생성 후에 경로 설정
    '''
    for file in glob(json_path+'/*.json'):
        with open(file, 'r') as f:
            annotations = json.load(f)
        annotation = annotations['annotations']
        for bbox in annotation:
            x_min, x_max = (bbox['points'][0][0]), (bbox['points'][2][0])
            y_min, y_max = (bbox['points'][0][1]), (bbox['points'][2][1])
            label = bbox['class']

            filename = annotations['filename']+'.txt'
            if not os.path.exists(f"{mov_path}/{filename}"):
                f = open(f"{mov_path}/{filename}",'w') 
                f.close()
            with open(f"{mov_path}/{filename}", 'a') as f: 
                    f.writelines(f"{label} {x_min} {y_min} {x_max} {y_max}\n")

In [11]:
def convert_aimmo_to_coco(
    save_name = 'data',
    data_path = 'C:/Users/Lee/Desktop/AIFFEL/Test/annotation' ,
    save_path = 'C:/Users/Lee/Desktop/AIFFEL/Test/'):
    
    # first check
    # !pip3 install numpyencoder
    # 
    res ={
        'images':[],
        'annotations':[],
        'categories':[]
    }
    categories = {
        'car' : 0,
        'bus' : 1,
        'truck' : 2,
        'pedestrian':3,
    }

    obj_count = 0

    for index, file in enumerate(glob(os.path.join(data_path,'*.json'))):
        with open(file,'r',encoding='utf-8') as annotations:
            annotation = json.load(annotations)

            # images 
            res['images'].append({
                'file_name':annotation['filename'],
                'width': 1920,
                'height': 1024,
                'id': index
            })

            # annotations
            for anno in annotation['annotations']:
                box = np.array(anno['points'])
                x1 , y1 ,x2 ,y2  = min(box[:, 0], ), min(box[: , 1]) , max(box[: , 0]) , max(box[:, 1])
                w, h = x2 - x1 , y2 - y1
                if anno['class']!= 'other':
                    res['annotations'].append({
                        'bbox' : [x1,y1,w,h],
                        'image_id': index,
                        'area' : anno['area'],
                        'iscrowd' : 0,
                        'category_id' : categories[anno['class']],
                        'id' : obj_count
                    })
                obj_count += 1
    # categoories
    for key,value in categories.items():
        res['categories'].append({'id':value  , 'name': key})

    with open(save_path + f'COCO_format_{save_name}.json','w') as file :
        json.dump(res,file,cls = NumpyEncoder , indent ='\t')

In [12]:
def make_txt(png_list,move_path,file_name):
    for txt in png_list:
        if txt.endswith('txt'):
            with open(move_path+f'/{file_name}.txt','a') as f:
                f.writelines(f"{txt}\n")

# Convert

In [14]:
data_path = 'C:/Users/Lee/Desktop/AIFFEL/Test/annotation'

In [15]:
number_of_data(data_path)

(0, 800, 800)

In [16]:
annotations = get_annotation(data_path)

In [16]:
size, label ,attribute = annotation_summary(annotations)

size of image {'1920*1024'}
number of label 11
number of class 18
number of sunny and day image: 396
number of train annotation: 396
number of validation annotation: 0
number of test annotation: 0


In [49]:
move_list = move_file(data_path)

In [13]:
data_path = 'C:/Users/Lee/Desktop/AIFFEL/Test'

In [61]:
change_file_name(data_path)

In [64]:
adjust_annotation(data_path)

0

In [67]:
make_class(data_path)

0

In [72]:
root_path = 'C:/Users/Lee/Desktop/AIFFEL/Test/'
image_path = 'C:/Users/Lee/Desktop/AIFFEL/Test/image/'
annotation_path = 'C:/Users/Lee/Desktop/AIFFEL/Test/annotation/'
split_image_annotation(root_path,image_path,annotation_path)

In [80]:
convert_aimmo_to_coco()

In [85]:
json_to_txt()

In [90]:
png = os.listdir('C:/Users/Lee/Desktop/AIFFEL/Test/annotation_txt')
test_png = random.choice(png, size = len(png), replace=False)

In [92]:
make_txt(test_png,move_path='C:/Users/Lee/Desktop/AIFFEL/Test',file_name='middle_test')

# test annotation

In [24]:
c = 0
list_name = []
for annotation in annotations:
    if annotation['weather'] == 'sunny' and annotation['time'] == 'day':
        c+=1
        list_name.append(annotation['filename'])

In [38]:
for i in list_name:
    shutil.copy(f'C:/Users/Lee/Desktop/AIFFEL/Test/annotation/{i}',f'C:/Users/Lee/Desktop/AIFFEL/Test/sun_day_annotation')

In [52]:
convert_aimmo_to_coco()

In [28]:
make_txt(list_name,move_path='C:/Users/Lee/Desktop/AIFFEL/Test',file_name='middle_sunday_test')

In [1]:
def make_txt(png_list,move_path,file_name):
    for txt in png_list:
        if txt.endswith('txt'):
            with open(move_path+f'/{file_name}.txt','a') as f:
                f.writelines(f"{txt}\n")